---
format:
    html:
        embed-resources: true
---

---
format:
    html:
        embed-resources: true
---

# Optional Bonus: Resume and cover letter creator 

This is not required, but will earn you up to +5 bonus points of extra credit. 

Using a combination of an LLM API wrapper and Quarto, write a pipeline that will create a resume PDF (or HTML), and a cover letter, for each of the given job descriptions. 

If you are doing this, for privacy reasons, you MUST do it for a fake "Jane-doe" (e.g. fake education, fake experience, etc.). 

Here is something to get you started, it is partially complete and quite hacky.

https://jfh.georgetown.domains/centralized-lecture-content/content/general-resources/share/resume-tool/resume-tool%202.zip

In [2]:
import os
import json
import google.generativeai as genai
import time
import pandas as pd

with open('/Users/zp/Desktop/gemini.json', 'r') as f:
    data = json.load(f)
api_key = genai.configure(api_key= data['api_key'])


model = genai.GenerativeModel("gemini-1.5-flash")
json_folder_path = "/Users/zp/hw-3-zp199717"
save_path = "/Users/zp/hw-3-zp199717/data"

def gemini(description):
    prompt = """
    Clean and streamline job descriptions.
    Extract the following information from this job description in JSON format with explicit keys:
    please JSON format!!!!!

    {
        "Job Title": ...,
        "Company Name": ...,
        "Sector/Industry": ...,
        "Job Type": ...,
        "Experience Level": ...,
        "Education Level": ...,
        "Education Requirement": ...,
        "Skills/Technologies Required": ...,
        "Job Responsibilities/Duties": ...,
        "Summary": ...,
        "Required Years of Experience": ...,
        "Job Description Length": ...,
        "Certifications Required or Preferred": ...,
    }

    Job Description: """ + f'"{description}"'

    response = model.generate_content(prompt)
    json_text = response.candidates[0].content.parts[0].text
    json_text = json_text.strip("```json").strip("```").strip()
    return json.loads(json_text)

# Loop through JSON files and process job descriptions
all_processed_data = []
for filename in os.listdir(json_folder_path):
    if filename.endswith(".json"):
        json_file_path = os.path.join(json_folder_path, filename)
        
        with open(json_file_path, 'r') as f:
            content = json.load(f)
            jobs_data = content['jobs_results']
            
            for job in jobs_data:
                description = job.get("description", "")
                
                try:
                    job_features = gemini(description)
                    all_processed_data.append(job_features)
                    time.sleep(1)  
                except Exception as e:
                    # Skip on errors
                    continue  
    
    time.sleep(0.5)  


df = pd.DataFrame(all_processed_data)


def create_resume_qmd(job, job_id):
    resume_content = f"""
---
title: " trump's Resume"
date: "2024-11-08"
---

# Jane Doe
**{job['Job Title']}**  
{job['Summary']}

## Contact
- Email: trump@example.com
- Phone: (555) 123-4567
- Location: New York, NY
- LinkedIn: linkedin.com/in/donald.j.trump

## Work Experience
### {job.get("Job Title", "President of USA")}
- {job.get("Job Responsibilities/Duties", "Experience details relevant to this job.")}

## Education
**{job['Education Requirement']}**  
 University, New York, NY

## Skills
{job['Skills/Technologies Required']}
    """
    
    with open(f"resume_{job_id}.qmd", "w") as file:
        file.write(resume_content)

def create_cover_letter_qmd(job, job_id):
    cover_letter_content = f"""
---
title: "Cover Letter for {job['Job Title']}"
---

Dear Hiring Manager,

I am writing to express my interest in the {job['Job Title']} position at {job['Company Name']}. With a background in {job['Experience Level']}, I am excited to bring my skills in {job['Skills/Technologies Required']} to your team. I am particularly drawn to this role because {job['Summary'][:100]}.

Thank you for considering my application. I look forward to the opportunity to contribute to your organization.

Sincerely,  
donald.j.trump
    """
    
    with open(os.path.join(save_path, f"cover_letter_{job_id}.qmd"), "w") as file:
        file.write(cover_letter_content)



KeyboardInterrupt: 